In [ ]:
# -*- coding: utf-8 -*-
"""
IED por Entidad Federativa (últimos 4 trimestres)
- Fuente: Secretaría de Economía (datos.gob.mx / CKAN)
- Salida: Figura Plotly con estilo corporativo y estado resaltado + Streamlit Chart
"""
import streamlit as st
import io
import json
import math
import unicodedata
from datetime import datetime
from typing import Tuple

import pandas as pd
import requests
import plotly.graph_objects as go
import sys

# ==========================
# Configuración de estilo
# ==========================
DEFAULT_FONT = "Arial, sans-serif"
DEFAULT_BASE = "#0b132b"
DEFAULT_HIGHLIGHT = "#889064"
BG_COLOR = "white"
GRID_COLOR = "#d9d9d9"

FONT_PRIMARY = DEFAULT_FONT
COLOR_BASE = DEFAULT_BASE
COLOR_HIGHLIGHT = DEFAULT_HIGHLIGHT
COLOR_ACCENT = "#ff9f18"

# CAMBIO 1: Aumentamos la altura total para que quepan las etiquetas verticales
FIG_WIDTH = 1200
FIG_HEIGHT = 600 
TITLE = "Flujo de inversión extranjera"
SUBTITLE = "(Últimos 4 trimestres, Millones USD)"


# ==========================
# Utilidades (Sin cambios)
# ==========================
def normalize(s: str) -> str:
    if s is None: return ""
    s = s.strip()
    s = "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")
    return s.lower()

def map_estado_alias(estado: str) -> str:
    s = normalize(estado)
    aliases = {
        "cdmx": "Ciudad de México", "ciudad de mexico": "Ciudad de México", "distrito federal": "Ciudad de México",
        "edomex": "Estado de México", "mexico": "Estado de México", "coahuila de zaragoza": "Coahuila",
        "veracruz de ignacio de la llave": "Veracruz", "michoacan de ocampo": "Michoacán",
        "quintana roo": "Quintana Roo", "yucatan": "Yucatán", "nuevo leon": "Nuevo León",
        "san luis potosi": "San Luis Potosí", "baja california": "Baja California",
        "baja california sur": "Baja California Sur",
    }
    return aliases.get(s, estado.strip())

def ckan_package_show(dataset_id: str) -> dict:
    url = f"https://www.datos.gob.mx/api/3/action/package_show?id={dataset_id}"
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    data = r.json()
    if not data.get("success"): raise RuntimeError("CKAN devolvió success=False")
    return data["result"]

def pick_resource_url(resources: list) -> str:
    candidates = []
    for res in resources:
        fmt = (res.get("format") or "").lower()
        name = (res.get("name") or "") + " " + (res.get("description") or "")
        name_l = normalize(name)
        if fmt != "csv": continue
        score = 0
        if "entidad" in name_l: score += 2
        if "trimestre" in name_l or "trimestral" in name_l: score += 2
        if "tipo" in name_l: score += 1
        if score > 0:
            updated = res.get("last_modified") or res.get("revision_timestamp") or ""
            candidates.append((score, updated, res.get("url")))
    if not candidates:
        for res in resources:
            fmt = (res.get("format") or "").lower()
            name = (res.get("name") or "") + " " + (res.get("description") or "")
            name_l = normalize(name)
            if fmt == "csv" and "entidad" in name_l:
                updated = res.get("last_modified") or res.get("revision_timestamp") or ""
                candidates.append((1, updated, res.get("url")))
    if not candidates: raise RuntimeError("No encontré un recurso CSV adecuado.")
    candidates.sort(key=lambda x: (x[0], x[1]), reverse=True)
    return candidates[0][2]

def read_csv_robusto(url: str) -> pd.DataFrame:
    try: return pd.read_csv(url)
    except: pass
    try: return pd.read_csv(url, encoding="latin-1")
    except: pass
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    return pd.read_csv(io.BytesIO(r.content))

def detectar_columnas(df: pd.DataFrame) -> Tuple[str, str, str, str]:
    cols = {c.lower(): c for c in df.columns}
    col_entidad = next((cols[c] for c in cols if "entidad" in c), next((cols[c] for c in cols if "estado" in c), None))
    col_anio = next((cols[c] for c in cols if c in ("anio", "año", "year")), None)
    col_trim = next((cols[c] for c in cols if "trimestre" in c or c in ("trim", "quarter")), None)
    col_valor = None
    for key in ["usd", "mdd", "miles de dolares", "millones", "valor", "monto"]:
        col_valor = next((cols[c] for c in cols if key in c), None)
        if col_valor: break
    if col_valor is None:
        numeric_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
        for c in [col_anio, col_trim]: 
            if c in numeric_cols: numeric_cols.remove(c)
        col_valor = numeric_cols[0] if numeric_cols else None
    if not all([col_entidad, col_anio, col_trim, col_valor]):
        raise RuntimeError(f"Columnas no detectadas: {list(df.columns)}")
    return col_entidad, col_anio, col_trim, col_valor

def rolling_4t(df, col_entidad, col_anio, col_trim, col_valor):
    df = df.copy()
    df[col_trim] = df[col_trim].astype(str).str.extract(r"(\d+)").astype(int)
    df[col_anio] = df[col_anio].astype(int)
    if df.empty: return pd.DataFrame(), ""
    max_year = df[col_anio].max()
    max_q = df.loc[df[col_anio] == max_year, col_trim].max()
    df["_t"] = df[col_anio] * 4 + df[col_trim]
    t_max = max_year * 4 + max_q
    window = {t_max, t_max - 1, t_max - 2, t_max - 3}
    df_4t = df[df["_t"].isin(window)].copy()
    g = df_4t.groupby(col_entidad, as_index=False)[col_valor].sum()
    g.rename(columns={col_valor: "mdd_4t"}, inplace=True)
    g.sort_values("mdd_4t", ascending=False, inplace=True)
    g["rank"] = range(1, len(g) + 1)
    return g, f"{max_year}T{max_q}"

def formatea_mdd(x): return f"${abs(x):,.0f} M USD"

def escala_y_publicacion(valores):
    vmin, vmax = float(min(valores)), float(max(valores))
    y_min = -500 if vmin < 0 else 0
    y_max = int((math.ceil(vmax / 500.0)) * 500) + 500
    return y_min, y_max, y_min, 1000

# ==========================
# CONSTRUCCIÓN DE FIGURA
# ==========================
def construir_figura(df_rank: pd.DataFrame, estado_sel: str, periodo: str) -> go.Figure:
    global FONT_PRIMARY, COLOR_BASE, COLOR_HIGHLIGHT, BG_COLOR, GRID_COLOR, TITLE, SUBTITLE, FIG_WIDTH, FIG_HEIGHT

    estados = df_rank.iloc[:, 0].tolist()
    valores = df_rank["mdd_4t"].tolist()
    colors = [COLOR_BASE] * len(estados)
    estado_norm = normalize(estado_sel)
    match_idx = None

    for i, e in enumerate(estados):
        if normalize(e) == estado_norm:
            match_idx = i
            break
    if match_idx is None:
        estado_mapped = map_estado_alias(estado_sel)
        estado_norm2 = normalize(estado_mapped)
        for i, e in enumerate(estados):
            if normalize(e) == estado_norm2:
                match_idx = i
                estado_sel = estado_mapped
                break
    if match_idx is not None: colors[match_idx] = COLOR_HIGHLIGHT

    fig = go.Figure()
    fig.add_bar(
        x=estados, y=valores, marker_color=colors,
        hovertemplate="<b>%{x}</b><br>%{y:,.0f} M USD<extra></extra>",
    )
    y_min, y_max, tick0, dtick = escala_y_publicacion(valores)

    # =========================================================================
    # AJUSTE DE LAYOUT PARA EVITAR TRUNCAMIENTO
    # =========================================================================
    # Definimos un margen inferior (b) muy grande (210px) para acomodar:
    # 1. Los nombres de estados verticales (aprox 150px para nombres largos)
    # 2. Espacio extra
    # 3. La fuente al final
    margin_bottom = 210
    
    # Calcular posición relativa Y de la anotación
    # En Plotly 'paper', 0 es el eje X. Los valores negativos van hacia el margen.
    # Si el plot area es (height - margin_top - margin_bottom), un valor de -0.45 
    # baja lo suficiente sin salir del container global, siempre que el margen exista.
    
    fig.update_layout(
        width=FIG_WIDTH,
        height=FIG_HEIGHT, # 600px
        plot_bgcolor=BG_COLOR,
        paper_bgcolor=BG_COLOR,
        font=dict(family=FONT_PRIMARY, size=16, color="#111"),
        # CAMBIO 2: Margen inferior explícito y grande
        margin=dict(l=60, r=40, t=70, b=margin_bottom), 
        title=dict(
            text=f"{TITLE}<br><span style='font-size:14px;color:#666;'>{SUBTITLE} · {periodo}</span>",
            x=0.5, xanchor="center", font=dict(family=FONT_PRIMARY)
        ),
        xaxis=dict(
            tickangle=90, # Vertical
            showgrid=False,
            tickfont=dict(size=12, family=FONT_PRIMARY),
        ),
        yaxis=dict(
            title=None, showgrid=True, gridcolor=GRID_COLOR, zeroline=True,
            zerolinecolor="#bbb", tickformat=",.0f", range=[y_min, y_max],
            tick0=tick0, dtick=dtick, tickfont=dict(family=FONT_PRIMARY)
        ),
    )

    # Box superior derecha
    if match_idx is not None:
        estado_box, valor_box, color_box = estados[match_idx], valores[match_idx], COLOR_HIGHLIGHT
    else:
        estado_box, valor_box, color_box = estados[0], valores[0], COLOR_BASE

    fig.add_annotation(
        x=0.92, y=0.93, xref="paper", yref="paper",
        text=f"<b>{estado_box}</b><br>{formatea_mdd(valor_box)}",
        showarrow=False, align="center",
        font=dict(color="#444", size=14, family=FONT_PRIMARY),
        bordercolor="rgba(0,0,0,0)", borderpad=8, bgcolor="rgba(255,255,255,0.25)"
    )
    fig.add_shape(
        type="rect", xref="paper", yref="paper", x0=0.78, x1=0.98, y0=0.74, y1=0.92,
        line=dict(color=color_box, width=1, dash="dot"), fillcolor="rgba(0,0,0,0)", layer="above"
    )

    # =========================================================================
    # CAMBIO 3: Anotación de Fuente con posición calibrada
    # =========================================================================
    # y=-0.45 funciona bien con un margen de 210px y altura de 600px.
    # align="left" y xanchor="left" aseguran que empiece a la izquierda.
    fig.add_annotation(
        text="Fuente: Secretaría de Economía (datos.gob.mx)",
        xref="paper", 
        yref="paper",
        x=0, 
        y=-0.45,  # Baja lo suficiente para librar "Veracruz..." y "Michoacán..."
        showarrow=False,
        xanchor='left',
        yanchor='bottom', # Anclar al fondo del texto
        font=dict(size=11, color="gray", family=FONT_PRIMARY)
    )

    return fig

def main():
    global FONT_PRIMARY, COLOR_BASE, COLOR_HIGHLIGHT
    try:
        FONT_PRIMARY = active_font
        COLOR_HIGHLIGHT = active_palette[0]
        COLOR_BASE = active_palette[1] if len(active_palette) > 1 else DEFAULT_BASE
    except NameError: pass

    estado_input = None
    try: estado_input = input().strip()
    except: pass
    if not estado_input:
        estado_input = "Aguascalientes"
        st.info(f"Usando '{estado_input}' como valor por defecto.")

    DATASET_ID = "inversion_extranjera_directa"
    try:
        pkg = ckan_package_show(DATASET_ID)
        resource_url = pick_resource_url(pkg["resources"])
        df_raw = read_csv_robusto(resource_url)
        col_entidad, col_anio, col_trim, col_valor = detectar_columnas(df_raw)
        df = df_raw.dropna(subset=[col_valor])
        df_rank, periodo = rolling_4t(df, col_entidad, col_anio, col_trim, col_valor)

        if df_rank.empty:
            st.error("Dataset vacío o inválido.")
        else:
            fig = construir_figura(df_rank, map_estado_alias(estado_input), periodo)
            st.plotly_chart(fig, use_container_width=True)

    except Exception as e:
        st.error(f"Error: {e}")

if __name__ == "__main__":
    main()

Escribe el nombre del Estado a resaltar (p. ej., 'Hidalgo', 'CDMX', 'Estado de México'): Hidalgo


2025-11-30 23:43:37.935 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
2025-11-30 23:43:37.937 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-30 23:43:37.938 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-30 23:43:37.939 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-30 23:43:38.007 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-11-30 23:43:38.008 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-30 23:43:38.009 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare 